In [2]:
import os
import shutil
from kaggle.api.kaggle_api_extended import KaggleApi

# Configuraciòn para descargar el dataset de kaggle
kaggle_json_path = 'kaggle.json'

if not os.path.exists(os.path.expanduser('~/.kaggle')):
    os.makedirs(os.path.expanduser('~/.kaggle'))

shutil.move(kaggle_json_path, os.path.expanduser('~/.kaggle/kaggle.json'))

os.chmod(os.path.expanduser('~/.kaggle/kaggle.json'), 0o600)

api = KaggleApi()
api.authenticate()

# Descargar y descomprimir el dataset de Kaggle
dataset = 'gomezp/benchmark-for-automatic-glottis-segmentation'
api.dataset_download_files(dataset, path='.', unzip=True)

print("Descarga completada.")


Dataset URL: https://www.kaggle.com/datasets/gomezp/benchmark-for-automatic-glottis-segmentation
Descarga completada.


In [6]:
import cv2

# Se definen los directorios de imágenes segmentadas y originales, y el directorio de salida
seg_image_dir = '/home/voicelab/Desktop/segmentation_glottis/BAGLS/test/test'
image_dir = '/home/voicelab/Desktop/segmentation_glottis/BAGLS/test/test'
output_dir = '/home/voicelab/Desktop/segmentation_glottis/YOLOV8/val'  

os.makedirs(output_dir, exist_ok=True)

# Se procesa cada imagen en el directorio de imágenes segmentadas
for image_file in os.listdir(seg_image_dir):
    if image_file.endswith('_seg.png'): # Se identifican mascaras que estan nombradas con _seg
        seg_image_path = os.path.join(seg_image_dir, image_file)
        seg_image = cv2.imread(seg_image_path, cv2.IMREAD_GRAYSCALE)

        # Encontrar los contornos en la imagen segmentada
        contours, _ = cv2.findContours(seg_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        height, width = seg_image.shape
        original_image_file = image_file.replace('_seg.png', '.png')
        label_file = os.path.join(output_dir, original_image_file.replace('.png', '.txt'))

        # Se escriben los contornos normalizados en un archivo de etiquetas
        with open(label_file, 'w') as f:
            for contour in contours:
                contour = contour.flatten()
                normalized_contour = [str(coord / width) if i % 2 == 0 else str(coord / height) for i, coord in enumerate(contour)]
                line = '0 ' + ' '.join(normalized_contour) + '\n'
                f.write(line)

        # Copiar la imagen original al directorio de salida
        original_image_path = os.path.join(image_dir, original_image_file)
        
        shutil.copy(original_image_path, os.path.join(output_dir, original_image_file))

print("Conversión completada.")


Conversión completada.


In [8]:
# Directorios
dataset_dir = '/home/voicelab/Desktop/segmentation_glottis/YOLOV8/'
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

new_image_train_dir = os.path.join(dataset_dir, 'dataset/images/train')
new_image_val_dir = os.path.join(dataset_dir, 'dataset/images/val')
new_label_train_dir = os.path.join(dataset_dir, 'dataset/labels/train')
new_label_val_dir = os.path.join(dataset_dir, 'dataset/labels/val')

# Creacion de nuevas carpetas
os.makedirs(new_image_train_dir, exist_ok=True)
os.makedirs(new_image_val_dir, exist_ok=True)
os.makedirs(new_label_train_dir, exist_ok=True)
os.makedirs(new_label_val_dir, exist_ok=True)

# Se mueven los archivos de imagen y etiqueta de entrenamiento a los nuevos directorios
for file_name in os.listdir(train_dir):
    file_path = os.path.join(train_dir, file_name)
    if file_name.endswith('.png'):
        shutil.move(file_path, os.path.join(new_image_train_dir, file_name))
    elif file_name.endswith('.txt'):
        shutil.move(file_path, os.path.join(new_label_train_dir, file_name))

# Se mueven los archivos de imagen y etiqueta de validaciòn a los nuevos directorios
for file_name in os.listdir(val_dir):
    file_path = os.path.join(val_dir, file_name)
    if file_name.endswith('.png'):
        shutil.move(file_path, os.path.join(new_image_val_dir, file_name))
    elif file_name.endswith('.txt'):
        shutil.move(file_path, os.path.join(new_label_val_dir, file_name))

print("Archivos reorganizados exitosamente.")


Archivos reorganizados exitosamente.


In [14]:
# Verificaciòn de que los labels contengan el minimo de coordenadas pR el formato yolo y correciòn
label_dirs = ['/home/voicelab/Desktop/segmentation_glottis/YOLOV8/dataset/labels/train', '/home/voicelab/Desktop/segmentation_glottis/YOLOV8/dataset/labels/val']

for label_dir in label_dirs:
    for label_file in os.listdir(label_dir):
        label_path = os.path.join(label_dir, label_file)
        
        with open(label_path, 'r') as file:
            lines = file.readlines()
        
        has_invalid_lines = False
        for line in lines:
            parts = line.strip().split()
            # Verificar que la línea tenga al menos 7 columnas (1 clase + 3 pares de coordenadas)
            if len(parts) < 7 or (len(parts) - 1) % 2 != 0:
                has_invalid_lines = True
                print(f"Invalid line found in {label_path}: {line.strip()}")
                break
        
        # Si se encontró alguna línea inválida se vacia el archivo
            with open(label_path, 'w') as file:
                pass  

print("Verificación y corrección de etiquetas completada.")


Invalid line found in /home/voicelab/Desktop/segmentation_glottis/YOLOV8/dataset/labels/train/18370.txt: 0 0.4296875 0.568359375 0.4296875 0.572265625
Invalid line found in /home/voicelab/Desktop/segmentation_glottis/YOLOV8/dataset/labels/train/54954.txt: 0 0.5546875 0.525390625 0.5546875 0.548828125
Invalid line found in /home/voicelab/Desktop/segmentation_glottis/YOLOV8/dataset/labels/train/25262.txt: 0 0.68359375 0.665625 0.68359375 0.678125
Invalid line found in /home/voicelab/Desktop/segmentation_glottis/YOLOV8/dataset/labels/train/49077.txt: 0 0.4951923076923077 0.8494318181818182 0.4951923076923077 0.8522727272727273
Invalid line found in /home/voicelab/Desktop/segmentation_glottis/YOLOV8/dataset/labels/train/26325.txt: 0 0.4296875 0.45703125
Invalid line found in /home/voicelab/Desktop/segmentation_glottis/YOLOV8/dataset/labels/train/30456.txt: 0 0.5576923076923077 0.42329545454545453
Invalid line found in /home/voicelab/Desktop/segmentation_glottis/YOLOV8/dataset/labels/train/